In [168]:
pip install tabulate


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sqlite3
import pandas as pd
import os
from tabulate import tabulate
from datetime import datetime

In [2]:
# Dynamically determine the base project root directory
current_dir = os.getcwd()  # Current working directory (from which the notebook is run)
project_root = os.path.abspath(os.path.join(current_dir, '..','Healthcare_ETL_Project'))

# Dynamically locate the database path in the 'db' folder within the project root
db_dir = os.path.join(project_root, "db")
db_path = os.path.join(db_dir, "healthcare_data.db")

# Ensure the database exists at the expected location
if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database not found at {db_path}")

# Connect to the SQLite database
conn = sqlite3.connect(db_path)
print(f"Database connection successful: {db_path}")


Database connection successful: /Users/avinashmacbookair/Documents/TREND_Health_Partners_3/Healthcare_ETL_Project/db/healthcare_data.db


In [3]:
# Example query: Fetching data from TREATMENT table
query = "SELECT * FROM PROVIDER;"  # Adjust query as needed
df = pd.read_sql_query(query, conn)

# Display the result as a nicely formatted table
if not df.empty:
    print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))
else:
    print("No data returned from the query.")

+--------------+---------------+--------------+-------------+-----------------+-------------------+---------------------------------------+--------------+------------+--------------+
|   Version_ID |   Provider_ID | First_Name   | Last_Name   |   Speciality_Id | Speciality_Name   | Affiliated_Hospital                   | Valid_From   | Valid_To   |   Is_Current |
|--------------+---------------+--------------+-------------+-----------------+-------------------+---------------------------------------+--------------+------------+--------------|
|            1 |             1 | Nandini      | Srivastava  |               8 | Radiology         | Mayo Clinic                           | 2025-04-29   |            |            1 |
|            2 |             2 | Aashi        | Devi        |               1 | Cardiology        | Cleveland Clinic                      | 2025-04-29   |            |            1 |
|            3 |             3 | Madhavi      | Ahluwalia   |              12 | Nephr

In [5]:
cursor = conn.cursor()

cursor.execute("""
    UPDATE PROVIDER 
    SET Affiliated_Hospital = 'New York-Presbyterian Hospital'
    WHERE Provider_ID = 5;
""")

cursor.execute("""
    UPDATE PROVIDER 
    SET Affiliated_Hospital = 'Cleveland Clinic'
    WHERE Provider_ID = 6;
""")

cursor.execute("""
    UPDATE PROVIDER 
    SET Affiliated_Hospital = 'Johns Hopkins Hospital'
    WHERE Provider_ID = 7;
""")

cursor.execute("""
    UPDATE PROVIDER 
    SET Affiliated_Hospital = 'Massachusetts General Hospital'
    WHERE Provider_ID = 8;
""")


conn.commit()

df = pd.read_sql_query("SELECT * FROM PROVIDER", conn)
print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))


+--------------+---------------+--------------+-------------+-----------------+-------------------+---------------------------------------+---------------------+---------------------+--------------+
|   Version_ID |   Provider_ID | First_Name   | Last_Name   |   Speciality_Id | Speciality_Name   | Affiliated_Hospital                   | Valid_From          | Valid_To            |   Is_Current |
|--------------+---------------+--------------+-------------+-----------------+-------------------+---------------------------------------+---------------------+---------------------+--------------|
|            1 |             1 | Nandini      | Srivastava  |               8 | Radiology         | Mayo Clinic                           | 2025-04-29          |                     |            1 |
|            2 |             2 | Aashi        | Devi        |               1 | Cardiology        | Cleveland Clinic                      | 2025-04-29          |                     |            1 |
|    

In [6]:

query = "SELECT Count(*) AS Total_Records FROM TREATMENT LIMIT 10;"  
df = pd.read_sql_query(query, conn)

if not df.empty:
    print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))
else:
    print("No data returned from the query.")

+-----------------+
|   Total_Records |
|-----------------|
|          350000 |
+-----------------+


In [44]:
# 1.Calculate the average treatment duration per treatment type
query = """
WITH TreatmentSummary AS (
    SELECT Type, 
           AVG(Treatment_Duration) AS Avg_Treatment_Duration
    FROM TREATMENT
    GROUP BY Type
)
SELECT * FROM TreatmentSummary;
"""
df = pd.read_sql_query(query, conn)

if not df.empty:
    print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))
else:
    print("No data returned from the query.")


+-----------------+--------------------------+
| Type            |   Avg_Treatment_Duration |
|-----------------+--------------------------|
| pharmacological |                  4.48511 |
| preventive      |                  4.49862 |
| surgical        |                  4.49565 |
| therapeutic     |                  4.49916 |
+-----------------+--------------------------+


In [ ]:
# 2.Find the most common outcome day of the week for treatments
query = """
WITH TreatmentDays AS (
    SELECT Outcome_Day, 
           COUNT(*) AS Frequency
    FROM TREATMENT
    GROUP BY Outcome_Day
)
SELECT Outcome_Day
FROM TreatmentDays
ORDER BY Frequency DESC
LIMIT 1;
"""
df = pd.read_sql_query(query, conn)

if not df.empty:
    print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))
else:
    print("No data returned from the query.")


+---------------+
| Outcome_Day   |
|---------------|
| Monday        |
+---------------+


In [48]:
# 3.Total number of treatments and the average treatment cost for each provider
query = """
WITH ProviderTreatmentStats AS (
    SELECT Provider_ID, 
           COUNT(*) AS Treatment_Count, 
           AVG(Cost) AS Avg_Cost
    FROM TREATMENT
    GROUP BY Provider_ID
)
SELECT * FROM ProviderTreatmentStats;
"""
df = pd.read_sql_query(query, conn)

if not df.empty:
    print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))
else:
    print("No data returned from the query.")


+---------------+-------------------+------------+
|   Provider_ID |   Treatment_Count |   Avg_Cost |
|---------------+-------------------+------------|
|             1 |             46720 |    29701.8 |
|             2 |             46793 |    29642.4 |
|             3 |             46412 |    29700   |
|             4 |             46719 |    29750   |
|             5 |             46635 |    29707.4 |
|             6 |             46521 |    29658.9 |
|             7 |             46887 |    29715.4 |
|             8 |             23313 |    29529.2 |
+---------------+-------------------+------------+


In [49]:
# 4.Calculate average cost per month
query = """
WITH MonthlyCost AS (
    SELECT 
        strftime('%Y', Outcome_Date) AS Year,
        strftime('%m', Outcome_Date) AS Month,
        AVG(Cost) AS Average_Cost
    FROM TREATMENT
    WHERE Year IN ('2024', '2025')
    GROUP BY Year, Month
    ORDER BY Year, Month
)
SELECT * FROM MonthlyCost;
"""

df = pd.read_sql_query(query, conn)

# Format the Average_Cost column to show full precision with commas
df['Average_Cost'] = df['Average_Cost'].apply(lambda x: '{:,.2f}'.format(x))

if not df.empty:
    print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))
else:
    print("No data returned from the query.")


+--------+---------+----------------+
|   Year |   Month | Average_Cost   |
|--------+---------+----------------|
|   2024 |      01 | 29,672.92      |
|   2024 |      02 | 29,696.48      |
|   2024 |      03 | 29,666.31      |
|   2024 |      04 | 29,765.95      |
|   2024 |      05 | 29,646.44      |
|   2024 |      06 | 29,716.85      |
|   2024 |      07 | 29,516.85      |
+--------+---------+----------------+


In [7]:

# 5.How many doctors have changed their affiliated hospital in the last 6 months
query = """
SELECT COUNT(DISTINCT p.Provider_ID) AS doctors_changed_hospital
FROM (
    SELECT Provider_ID
    FROM PROVIDER
    GROUP BY Provider_ID
    HAVING COUNT(DISTINCT Affiliated_Hospital) > 1
) AS changed_providers
JOIN PROVIDER p ON changed_providers.Provider_ID = p.Provider_ID
WHERE p.Valid_From >= date('now', '-6 months');
"""

df = pd.read_sql_query(query, conn)

if not df.empty:
    print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))
else:
    print("No data returned from the query.")


+----------------------------+
|   doctors_changed_hospital |
|----------------------------|
|                          4 |
+----------------------------+


In [8]:

# 6.Details of doctors have changed their affiliated hospital in the last 6 months
query = """
SELECT DISTINCT p.Provider_ID, p.First_Name, p.Last_Name
FROM (
    SELECT Provider_ID
    FROM PROVIDER
    GROUP BY Provider_ID
    HAVING COUNT(DISTINCT Affiliated_Hospital) > 1
) AS changed_providers
JOIN PROVIDER p ON changed_providers.Provider_ID = p.Provider_ID
WHERE p.Valid_From >= date('now', '-6 months');
"""

df = pd.read_sql_query(query, conn)

if not df.empty:
    print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))
else:
    print("No data returned from the query.")


+---------------+--------------+-------------+
|   Provider_ID | First_Name   | Last_Name   |
|---------------+--------------+-------------|
|             5 | Rudra        | Gandhi      |
|             6 | Leela        | Jha         |
|             7 | Pratibha     | Yadav       |
|             8 | Sarika       | Zutshi      |
+---------------+--------------+-------------+


In [ ]:

# 7.Top 5 Providers with Highest Treatment Effectiveness
query = """
SELECT Provider_ID, AVG(Effectiveness_Score) AS Avg_effectiveness
FROM TREATMENT
GROUP BY Provider_ID
ORDER BY avg_effectiveness DESC
LIMIT 5;
"""

df = pd.read_sql_query(query, conn)

if not df.empty:
    print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))
else:
    print("No data returned from the query.")


+---------------+---------------------+
|   Provider_ID |   Avg_effectiveness |
|---------------+---------------------|
|             5 |             2.51221 |
|            10 |             2.51156 |
|            12 |             2.50496 |
|             1 |             2.50468 |
|            13 |             2.50292 |
+---------------+---------------------+


In [ ]:
# 8.Calculate the effectiveness score for treatments by disease
query = """
WITH EffectivenessByDisease AS (
    SELECT Disease_ID, 
           AVG(Effectiveness_Score) AS Avg_Effectiveness_Score
    FROM TREATMENT
    WHERE Effectiveness_Score IS NOT NULL
    GROUP BY Disease_ID
)
SELECT * FROM EffectivenessByDisease;
"""
df = pd.read_sql_query(query, conn)

if not df.empty:
    print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))
else:
    print("No data returned from the query.")


+--------------+---------------------------+
|   Disease_ID |   Avg_Effectiveness_Score |
|--------------+---------------------------|
|            1 |                   2.52023 |
|            2 |                   2.46752 |
|            3 |                   2.49717 |
|            4 |                   2.52146 |
|            5 |                   2.48843 |
|           16 |                   2.4925  |
|           17 |                   2.50139 |
|           18 |                   2.51407 |
|           19 |                   2.50972 |
|           20 |                   2.47337 |
|           21 |                   2.52607 |
|           22 |                   2.52652 |
|           23 |                   2.47625 |
|           24 |                   2.5005  |
|           25 |                   2.46881 |
|           26 |                   2.5102  |
|           27 |                   2.51076 |
|           28 |                   2.47    |
|           29 |                   2.47402 |
|         

In [ ]:
# 9.Determine the total cost of treatments by outcome quarter and year
query = """
WITH CostSummary AS (
    SELECT 
        strftime('%Y', Outcome_Date) AS Outcome_Year,
        Outcome_Quarter, 
        SUM(Cost) AS Total_Cost
    FROM TREATMENT
    GROUP BY Outcome_Year, Outcome_Quarter
)
SELECT * FROM CostSummary;
"""
df = pd.read_sql_query(query, conn)

# Set pandas options to avoid scientific notation globally
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Apply formatting to the Total_Cost column if needed
df['Total_Cost'] = df['Total_Cost'].apply(lambda x: '{:,.2f}'.format(x))

if not df.empty:
    print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))
else:
    print("No data returned from the query.")


+----------------+-------------------+------------------+
|   Outcome_Year |   Outcome_Quarter | Total_Cost       |
|----------------+-------------------+------------------|
|           2024 |                 1 | 4,667,369,635.31 |
|           2024 |                 2 | 5,185,229,884.95 |
|           2024 |                 3 | 5,238,164,852.77 |
|           2024 |                 4 | 5,244,743,843.22 |
|           2025 |                 1 | 455,790,336.80   |
+----------------+-------------------+------------------+


In [ ]:
conn.close() 